In [2]:
import os
import pandas as pd
from PIL import Image
from ultralytics import YOLO
import pytesseract
from pdf2image import convert_from_path
import subprocess
from pathlib import Path

In [3]:
# 모델 로딩
model = YOLO("./model/yolov11n-doclaynet.pt")

In [ ]:
# 클래스 매핑
LABEL_MAP = {
    'Text': 'text',
    'Title': 'title',
    'Section-header': 'subtitle',
    'Formula': 'equation',
    'Table': 'table',
    'Picture': 'image'
}

In [ ]:
def convert_to_images(input_path, temp_dir, dpi=200):
    ext = Path(input_path).suffix.lower()
    os.makedirs(temp_dir, exist_ok=True)

    if ext == ".pdf":
        return convert_from_path(input_path, dpi=dpi, output_folder=temp_dir, fmt="png")
    elif ext == ".pptx":
        # Convert pptx to pdf first
        subprocess.run([
            "libreoffice", "--headless", "--convert-to", "pdf", "--outdir", temp_dir, input_path
        ], check=True)
        pdf_path = os.path.join(temp_dir, Path(input_path).with_suffix(".pdf").name)
        return convert_from_path(pdf_path, dpi=dpi, output_folder=temp_dir, fmt="png")
    elif ext in [".jpg", ".jpeg", ".png"]:
        return [Image.open(input_path).convert("RGB")]
    else:
        raise ValueError(f"지원하지 않는 파일 형식입니다: {ext}")

In [ ]:
def scale_bbox_to_target(bbox, current_size, target_size):
    x1, y1, x2, y2 = bbox
    scale_x = target_size[0] / current_size[0]
    scale_y = target_size[1] / current_size[1]
    return [
        int(x1 * scale_x),
        int(y1 * scale_y),
        int(x2 * scale_x),
        int(y2 * scale_y)
    ]

In [ ]:
# OCR 정확도 높이는 방향으로 수정 요함
def extract_text(image_pil, bbox): 
    x1, y1, x2, y2 = bbox
    cropped = image_pil.crop((x1, y1, x2, y2))
    return pytesseract.image_to_string(cropped, lang='kor+eng').strip()

이 함수는 개선이 필요할까?
이 함수는 텍스트 추출의 기본 기능은 잘 수행하고 있습니다. 하지만 OCR의 정확도를 높이려면 다음 부분을 개선할 수 있습니다.

해상도 최적화: convert_to_images 함수에서 dpi 값을 조정하면 OCR 정확도가 달라질 수 있습니다. 예를 들어, DPI를 높이면 더 선명한 이미지를 얻어 정확도가 높아지지만, 처리 속도가 느려집니다.

전처리 강화: 텍스트 추출 전에 이미지의 대비(contrast)를 높이거나, 노이즈를 제거하는 등의 전처리 작업을 추가하면 OCR 성능이 더 좋아질 수 있습니다.

다른 OCR 엔진 사용: pytesseract 외에 EasyOCR나 PaddleOCR와 같은 다른 OCR 라이브러리를 사용해 보세요. 이들은 경우에 따라 더 높은 정확도를 제공할 수 있습니다.

결론적으로, 이 함수는 텍스트 추출의 핵심이므로, OCR 정확도를 높이는 방향으로 개선을 고려해 볼 수 있습니다.

In [ ]:
def inference_one_image(id_val, image_pil, target_size, conf_thres=0.3):
    original_size = image_pil.size  # (w, h)
    resized_image = image_pil.resize((1024, 1024))
    temp_path = "_temp_image.png"
    resized_image.save(temp_path)

    results = model(source=temp_path, imgsz=1024, conf=conf_thres, verbose=False)[0]
    os.remove(temp_path)

    predictions = []
    order = 0
    for box, score, cls in zip(results.boxes.xyxy, results.boxes.conf, results.boxes.cls):
        label = results.names[int(cls)]
        if label not in LABEL_MAP:
            continue
        category_type = LABEL_MAP[label]
        x1, y1, x2, y2 = scale_bbox_to_target(box.tolist(), (1024, 1024), target_size)
        text = extract_text(image_pil, [x1, y1, x2, y2]) if category_type in ['title', 'subtitle', 'text'] else ''
        predictions.append({
            'ID': id_val,
            'category_type': category_type,
            'confidence_score': score.cpu().item(),
            'order': order,
            'text': text,
            'bbox': f'{x1}, {y1}, {x2}, {y2}'
        })
        order += 1
    return predictions

네, 이 함수는 가장 많이 수정하고 개선해야 할 부분입니다. 특히 다음 두 가지가 핵심 개선점입니다.

순서(order) 예측 로직:

현재 코드는 order = 0으로 시작하여 루프를 돌 때마다 order += 1을 합니다. 이는 모델이 결과를 반환한 순서대로 순위를 매기는 것으로, 논리적인 읽기 순서와는 아무 관련이 없습니다.

개선 방법: 루프가 끝난 후 predictions 리스트를 bbox 좌표를 기준으로 재정렬하는 새로운 로직을 추가해야 합니다. 예를 들어, y1 좌표를 기준으로 오름차순 정렬하고, y1이 같을 경우 x1을 기준으로 다시 정렬하는 방식이 가장 기본적입니다.

수식(equation) 추출:

현재 코드에는 수식(Formula)이 category_type으로는 존재하지만, text를 추출하지 않습니다.

개선 방법: category_type이 'equation'일 때, text를 빈 문자열('')로 두는 대신, extract_text 함수와는 별개로 수식 이미지를 LaTeX로 변환하는 새로운 함수를 호출하여 결과를 text 필드에 채워 넣어야 합니다.

In [ ]:
def inference(test_csv_path="./data/test.csv", output_csv_path="./output/submission.csv"):
    output_dir = os.path.dirname(output_csv_path)
    os.makedirs(output_dir, exist_ok=True)

    temp_image_dir = "./temp_images"
    os.makedirs(temp_image_dir, exist_ok=True)

    csv_dir = os.path.dirname(test_csv_path)
    test_df = pd.read_csv(test_csv_path)
    all_preds = []

    for _, row in test_df.iterrows():
        id_val = row['ID']
        raw_path = row['path']
        file_path = os.path.normpath(os.path.join(csv_dir, raw_path))
        target_width = int(row['width'])
        target_height = int(row['height'])

        if not os.path.exists(file_path):
            print(f"⚠️ 파일 없음: {file_path}")
            continue

        try:
            images = convert_to_images(file_path, temp_image_dir)
            for i, image in enumerate(images):
                full_id = f"{id_val}_p{i+1}" if len(images) > 1 else id_val
                preds = inference_one_image(full_id, image, (target_width, target_height))
                all_preds.extend(preds)
            print(f"✅ 예측 완료: {file_path}")
        except Exception as e:
            print(f"❌ 처리 실패: {file_path} → {e}")

    result_df = pd.DataFrame(all_preds)
    result_df.to_csv(output_csv_path, index=False, encoding='UTF-8-sig')
    print(f"✅ 저장 완료: {output_csv_path}")

In [ ]:
if __name__ == "__main__":
    inference("./data/test.csv", "./output/submission.csv")